#Setup Library

In [ ]:
!pip -q install Sastrawi

In [ ]:
!pip -q install Sastrawi tqdm

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv("data_link_berita_with_content_cleaned_manual.csv")

# Lowercasing, Regex, Hapus Karakter Special, Punctuation Removal

In [ ]:
# 1) Load
df = pd.read_csv('/content/data_link_berita_with_content_cleaned_manual.csv')

# 2) Pola dasar: emoji/zero-width & hapus tanda baca
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"
    "\U0001F300-\U0001F5FF"
    "\U0001F680-\U0001F6FF"
    "\U0001F700-\U0001F77F"
    "\U0001F780-\U0001F7FF"
    "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FA6F"
    "\U0001FA70-\U0001FAFF"
    "\U00002702-\U000027B0"
    "\U000024C2-\U0001F251"
    "\U0001F1E6-\U0001F1FF"
    "]+"
)
ZW_CHARS = r"[\u200B-\u200D\uFEFF]"

# Hapus semua karakter non huruf/angka/spasi (tanda baca ikut hilang)
only_alnum_pattern = re.compile(r"[^0-9A-Za-zÀ-ÖØ-öø-ÿĀ-žḀ-ỿ\u0100-\u024F\u1E00-\u1EFF\s]")

# 3) Kamus/pola finansial yang ingin dipertahankan
CURRENCY = r"(?:rp|rupiah|usd|us|sgd|idr|eur|dollar|dolar|yen|cny|rmb)"
UNITS    = r"(?:ribu|jt|juta|m|miliar|milyar|bn|triliun|tn)"
PERCENT  = r"(?:%|persen)"

# Kata kunci finansial terdekat (sebelum/sesudah angka) yang membuat angka dipertahankan
# Kata kunci finansial (dengan variasi imbuhan umum)
FIN_KEYWORDS = (
    r"(?:"
    r"laba|rugi|untung|kerugian|keuntungan|pendapatan|revenue|penjualan|omzet|omset|biaya|beban|"
    r"aset|liabilitas|utang|piutang|ekuitas|arus|kas|cash|capital|kapitalisasi|valuasi|dividen|"
    r"ebitda|margin|subsidi|kupon|bunga|inflasi|deflasi|neraca|belanja|investasi|akuisisi|"
    r"penyertaan|buyback|rights\s*issue|ipo|obligasi|sukuk|deviden|capex|opex|laba\s*bersih|"
    # dinamika perubahan
    r"naik(?:nya|kan)?|kenaik(?:an|annya)?|turun(?:nya|kan)?|penurun(?:an|annya)?|"
    r"meningkat(?:kan)?|peningkat(?:an|annya)?|melemah(?:kan)?|pelemah(?:an|annya)?|"
    r"menguat(?:kan)?|penguat(?:an|annya)?|"
    r"lonjak(?:|an|annya)|melonjak|anjlok(?:|nya)|koreksi(?:|nya)|volatil(?:itas)?|"
    r"reli|rebound|rekor|tertinggi|terendah|all\s*time\s*high|all\s*time\s*low"
    r")"
)

# 4) Buat pola proteksi (lindungi dulu angka yang finansial)
protect_patterns = [
    re.compile(rf"\b{CURRENCY}\s*\d[\d\.]*\,?\d*\s*(?:{UNITS})?\b", flags=re.IGNORECASE),
    re.compile(rf"\b\d[\d\.]*\,?\d*\s*(?:{UNITS})\b", flags=re.IGNORECASE),
    re.compile(rf"\b\d[\d\.]*\,?\d*\s*(?:{PERCENT})\b", flags=re.IGNORECASE),
    # FIN_KEYWORDS di kiri/kanan angka (≤2 kata jarak)
    re.compile(rf"\b(?:{FIN_KEYWORDS})(?:\s+\w+){{0,2}}\s+\d[\d\.]*\,?\d*\b", flags=re.IGNORECASE),
    re.compile(rf"\b\d[\d\.]*\,?\d*(?:\s+\w+){{0,2}}\s+(?:{FIN_KEYWORDS})\b", flags=re.IGNORECASE),
]


# Placeholder unik agar aman saat remove
PH_L = "⟦KEEPNUM⟧"
PH_R = "⟧KEEPNUM⟦"

def protect_financial_numbers(text: str) -> str:
    # bungkus match dengan placeholder
    for pat in protect_patterns:
        text = pat.sub(lambda m: f"{PH_L}{m.group(0)}{PH_R}", text)
    return text

def clean_text_with_numeric_rules(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # normalisasi awal (hapus emoji, zero-width, tanda baca)
    text = emoji_pattern.sub(" ", text)
    text = re.sub(ZW_CHARS, "", text)
    text = only_alnum_pattern.sub(" ", text)

    # lowercase biar pola konsisten (opsional kalau sudah dilakukan di step 2)
    text = text.lower()

    # proteksi angka finansial
    text = protect_financial_numbers(text)

    # hapus semua digit yang tidak diproteksi
    text = re.sub(r"\d+", " ", text)

    # kembalikan placeholder -> konten asli
    text = text.replace(PH_L, "").replace(PH_R, "")

    # rapikan spasi
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 5) Terapkan ke kolom konten saja
df['konten_hapus_karakter'] = df['konten'].apply(clean_text_with_numeric_rules)

# 6) Simpan
df.to_csv('/content/garudaindonesia_news_lower_hapus_karakter.csv', index=False)

# 7) Cek contoh
df[['konten', 'konten_hapus_karakter']].head(5)


,konten,konten_hapus_karakter
0,Garuda Indonesia Kembali RUPSLB di Tengah Isu ...,garuda indonesia kembali rupslb di tengah isu ...
1,Garuda Gelar RUPSLB di Tengah Isu Masuknya Dir...,garuda gelar rupslb di tengah isu masuknya dir...
2,JAKARTA - Ketua Komisi V DPR Lasarus mengataka...,jakarta ketua komisi v dpr lasarus mengatakan ...
3,"Latar Belakang\nPada pertengahan 2023, wacana ...",latar belakang pada pertengahan wacana konsoli...
4,--\nPlt Menteri Badan Usaha Milik Negara (BUMN...,plt menteri badan usaha milik negara bumn seka...


# Normalisasi
Dilakukan menggunakan dataset sebagai berikut: https://github.com/nasalsabila/kamus-alay

In [ ]:
# --- 1) Load data ---
df = pd.read_csv('/content/garudaindonesia_news_lower_hapus_karakter.csv')  # punya kolom 'konten_lowercase'
lex = pd.read_csv('/content/colloquial-indonesian-lexicon.csv')  # punya kolom 'slang','formal'

# --- 2) Siapkan kamus slang->formal ---
# normalisasi kolom leksikon ke lowercase & strip
lex = lex[['slang', 'formal']].dropna()
lex['slang'] = lex['slang'].astype(str).str.lower().str.strip()
lex['formal'] = lex['formal'].astype(str).str.lower().str.strip()

# jika ada duplikat slang, ambil yang pertama
lex = lex.drop_duplicates(subset=['slang'], keep='first')

slang2formal = dict(zip(lex['slang'], lex['formal']))

# --- 3) Buat regex frasa/katanya (pakai boundary agar match utuh) ---
# urutkan kunci dari yang terpanjang agar frasa lebih panjang diprioritaskan
keys_sorted = sorted(slang2formal.keys(), key=len, reverse=True)

# escape semua kunci supaya karakter khusus aman
escaped = [re.escape(k) for k in keys_sorted if k]  # buang string kosong jika ada
pattern = re.compile(r'\b(?:' + '|'.join(escaped) + r')\b', flags=re.UNICODE)

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # substitusi berdasarkan kamus
    return pattern.sub(lambda m: slang2formal.get(m.group(0), m.group(0)), text)

# --- 4) Terapkan ke kolom konten_lowercase ---
df['konten_normalized'] = df['konten_hapus_karakter'].astype(str).apply(normalize_text)

# opsional: rapikan spasi ganda
df['konten_normalized'] = df['konten_normalized'].str.replace(r'\s+', ' ', regex=True).str.strip()

# --- 5) Simpan hasil ---
out_path = '/content/garudaindonesia_news_normalized.csv'
df.to_csv(out_path, index=False)

# --- 6) Cek contoh hasil ---
df[['konten_hapus_karakter', 'konten_normalized']].head(5)

,konten_hapus_karakter,konten_normalized
0,garuda indonesia kembali rupslb di tengah isu ...,garuda indonesia kembali rupslb di tengah isu ...
1,garuda gelar rupslb di tengah isu masuknya dir...,garuda gelar rupslb di tengah isu masuknya dir...
2,jakarta ketua komisi v dpr lasarus mengatakan ...,jakarta ketua komisi v dpr lasarus mengatakan ...
3,latar belakang pada pertengahan wacana konsoli...,latar belakang pada pertengahan wacana konsoli...
4,plt menteri badan usaha milik negara bumn seka...,plt menteri badan usaha milik negara bumn seka...


#Stopwords Removal

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# 1) Load hasil normalisasi
df = pd.read_csv('/content/garudaindonesia_news_normalized.csv')

# 2) Ambil daftar stopword Sastrawi (list → set untuk cepat)
factory = StopWordRemoverFactory()
stopwords = set(factory.get_stop_words())

# 4) (Opsional) Whitelist: kata yang TETAP dipertahankan walaupun ada di stopwords
whitelist = {
    'garuda','indonesia','maskapai','laba','rugi','untung','kerugian','keuntungan',
    'pendapatan','revenue','penjualan','omzet','omset','biaya','beban','aset',
    'liabilitas','utang','piutang','ekuitas','kas','cash','dividen','ebitda','margin',
    'subsidi','bunga','inflasi','deflasi','investasi','akuisisi','buyback','ipo',
    'obligasi','sukuk','capex','opex','rupiah','rp','usd','idr','sgd','miliar',
    'triliun','milyar','juta','persen','naik','turun','kenaikan','penurunan',
    'menguat','melemah','koreksi','anjlok','lonjak','reli','rebound'
}

# kalau kamu mau kosongkan whitelist, set() aja.

def remove_stopwords(text: str) -> str:
    if not isinstance(text, str):
        return ""
    tokens = text.split()
    kept = []
    for t in tokens:
        if t in whitelist:
            kept.append(t)            # whitelist menang
        elif t in stopwords:
            continue                  # buang stopword
        else:
            kept.append(t)
    return " ".join(kept)

# 5) Terapkan
df['konten_nostop'] = df['konten_normalized'].astype(str).apply(remove_stopwords)
df['konten_nostop'] = df['konten_nostop'].str.replace(r'\s+', ' ', regex=True).str.strip()

# 6) Simpan
df.to_csv('/content/garudaindonesia_news_nostop.csv', index=False)

# 7) Cek
df[['konten_normalized', 'konten_nostop']].head(5)

,konten_normalized,konten_nostop
0,garuda indonesia kembali rupslb di tengah isu ...,garuda indonesia rupslb tengah isu petinggi si...
1,garuda gelar rupslb di tengah isu masuknya dir...,garuda gelar rupslb tengah isu masuknya direks...
2,jakarta ketua komisi v dpr lasarus mengatakan ...,jakarta ketua komisi v dpr lasarus mengatakan ...
3,latar belakang pada pertengahan wacana konsoli...,latar belakang pertengahan wacana konsolidasi ...
4,plt menteri badan usaha milik negara bumn seka...,plt menteri badan usaha milik negara bumn seka...


#Stemming

In [ ]:
import time
from functools import lru_cache
from tqdm.auto import tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# 1) Load data
in_path  = '/content/garudaindonesia_news_nostop.csv'       # kolom: konten_nostop
out_path = '/content/garudaindonesia_news_stem.csv'
df = pd.read_csv(in_path)

# 2) Siapkan stemmer
stemmer = StemmerFactory().create_stemmer()

# 3) Fungsi stemming (pakai cache biar cepat)
@lru_cache(maxsize=200_000)
def stem_token(tok: str) -> str:
    return stemmer.stem(tok)

def stem_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return " ".join(stem_token(t) for t in text.split())

# 4) Konfigurasi progress bar (ETA tampil otomatis)
tqdm.pandas(desc="Stemming rows", unit="row", mininterval=0.5, dynamic_ncols=True)

start = time.perf_counter()

# 5) Apply dengan progress bar
df['konten_stem'] = df['konten_nostop'].astype(str).progress_apply(stem_text)

elapsed = time.perf_counter() - start

# 6) Rapikan & simpan
df['konten_stem'] = df['konten_stem'].str.replace(r'\s+', ' ', regex=True).str.strip()
df.to_csv(out_path, index=False)

print(f"Done. Rows: {len(df):,} | Elapsed: {elapsed:.2f}s (~{elapsed/60:.2f} min) | Saved to: {out_path}")

# 7) Cek contoh
df[['konten_nostop', 'konten_stem']].head(5)


Stemming rows:   0%|          | 0/469 [00:00<?, ?row/s]

Done. Rows: 469 | Elapsed: 511.85s (~8.53 min) | Saved to: /content/garudaindonesia_news_stem.csv


,konten_nostop,konten_stem
0,garuda indonesia rupslb tengah isu petinggi si...,garuda indonesia rupslb tengah isu petinggi si...
1,garuda gelar rupslb tengah isu masuknya direks...,garuda gelar rupslb tengah isu masuk direksi a...
2,jakarta ketua komisi v dpr lasarus mengatakan ...,jakarta ketua komisi v dpr lasarus kata dalam ...
3,latar belakang pertengahan wacana konsolidasi ...,latar belakang tengah wacana konsolidasi maska...
4,plt menteri badan usaha milik negara bumn seka...,plt menteri badan usaha milik negara bumn seka...
